<a href="https://colab.research.google.com/github/DarrenHsien/ML2021/blob/main/4_1_Phoneme_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Dataset : The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
https://www.youtube.com/watch?v=PdjXnQbu2zo

The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.
https://www.kaggle.com/c/ml2021spring-hw2/data

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data (#of training frames, 11xfeature dim)<br>
- `train_label_11.npy`: training label (framewise phoneme label)(0-38)<br>
- `test_11.npy`:  testing data (#of testing frames, 11xfeature dim)<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




Use skill
- Model architecture(layer? dimension? activation func?)
- Training(batch size? optimizer? learning rate? epoch?)
- Tips(batch norm? dropout? regularization?)


In [6]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
#!gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!unzip /content/drive/MyDrive/ColabNotebooks/timit_11.zip
!ls 

Archive:  /content/drive/MyDrive/ColabNotebooks/timit_11.zip
   creating: timit_11/
  inflating: timit_11/train_11.npy   
  inflating: timit_11/test_11.npy    
  inflating: timit_11/train_label_11.npy  
drive  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [9]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print(f'feature has {train.shape[1]} dim, and total have {train.shape[0]} frames')
print('Size of training label: {}'.format(train_label.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
feature has 429 dim, and total have 1229932 frames
Size of training label: (1229932,)
Size of testing data: (451552, 429)


## Create Dataset

In [10]:
import torch
from torch.utils.data import Dataset

# Crate a Dataset to implement in Dataloader
class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [11]:
# partial train val data
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [12]:
BATCH_SIZE = 64

from torch.utils.data import DataLoader

# Crate Dataloader with dataset implement
train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:9: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  if __name__ == '__main__':


Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [13]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

64

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [34]:
import torch
import torch.nn as nn

# dataloader iter batch shape : [64, 429]
# so the data input model first layer node input must equal to it   
# sigmod add after layer is use to help with gradient dscent and align data to --1~1
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.layer1 = nn.Linear(429, 1024)
        # add plus1
        self.bn1 = nn.BatchNorm1d(num_features=1024)
        self.layer2 = nn.Linear(1024, 512)
        # add plus1
        self.bn2 = nn.BatchNorm1d(num_features=512)
        self.layer3 = nn.Linear(512, 128)
        # add plus1
        self.bn3 = nn.BatchNorm1d(num_features=128)
        self.out = nn.Linear(128, 39)
        
        self.act_fn1 = nn.Sigmoid()
        self.act_fn2 = nn.ReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.bn1(x)
        x = self.act_fn2(x)

        x = self.layer2(x)
        x = self.bn2(x)
        x = self.act_fn2(x)

        x = self.layer3(x)
        x = self.bn3(x)
        x = self.act_fn2(x)

        x = self.out(x)
        
        return x

## Training

In [35]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [36]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [37]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 20               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model1.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

DEVICE: cuda


In [38]:
# start training
# inputs shape : [64, 429]
# labels shape : [64]
# outputs shape : [64, 39]
# train_pred shape : [64]
best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs)
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 
        
        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')



[001/020] Train Acc: 0.636574 Loss: 1.194184 | Val Acc: 0.694374 loss: 0.954806
saving model with acc 0.694
[002/020] Train Acc: 0.701519 Loss: 0.925820 | Val Acc: 0.706919 loss: 0.903160
saving model with acc 0.707
[003/020] Train Acc: 0.725068 Loss: 0.841142 | Val Acc: 0.715538 loss: 0.874767
saving model with acc 0.716
[004/020] Train Acc: 0.742289 Loss: 0.781045 | Val Acc: 0.719022 loss: 0.863454
saving model with acc 0.719
[005/020] Train Acc: 0.756566 Loss: 0.732285 | Val Acc: 0.718697 loss: 0.866900
[006/020] Train Acc: 0.768515 Loss: 0.692473 | Val Acc: 0.720770 loss: 0.866015
saving model with acc 0.721
[007/020] Train Acc: 0.779066 Loss: 0.657511 | Val Acc: 0.716412 loss: 0.881633
[008/020] Train Acc: 0.788447 Loss: 0.625214 | Val Acc: 0.717631 loss: 0.887415
[009/020] Train Acc: 0.797039 Loss: 0.598500 | Val Acc: 0.715961 loss: 0.902041
[010/020] Train Acc: 0.804436 Loss: 0.573156 | Val Acc: 0.712501 loss: 0.920026
[011/020] Train Acc: 0.811542 Loss: 0.550772 | Val Acc: 0.71

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [22]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [23]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))